In [ ]:
#!pip install keras
#For one time only

In [ ]:
#!pip install tensorflow
#For one time only

### 1. Importing Libraries

In [1]:
import numpy
from keras.datasets import imdb #datset provided by Stanford
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(1234)

Using TensorFlow backend.


### 2. Loading dataset

In [2]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

### 3. Analyse Datset tounderstand it

In [7]:
X_train.shape,y_train.shape

((25000,), (25000,))

In [10]:
X_train[0]#first record/row

[1,
 14,
 22,
 16,
 43,
 530,
 973,
 1622,
 1385,
 65,
 458,
 4468,
 66,
 3941,
 4,
 173,
 36,
 256,
 5,
 25,
 100,
 43,
 838,
 112,
 50,
 670,
 2,
 9,
 35,
 480,
 284,
 5,
 150,
 4,
 172,
 112,
 167,
 2,
 336,
 385,
 39,
 4,
 172,
 4536,
 1111,
 17,
 546,
 38,
 13,
 447,
 4,
 192,
 50,
 16,
 6,
 147,
 2025,
 19,
 14,
 22,
 4,
 1920,
 4613,
 469,
 4,
 22,
 71,
 87,
 12,
 16,
 43,
 530,
 38,
 76,
 15,
 13,
 1247,
 4,
 22,
 17,
 515,
 17,
 12,
 16,
 626,
 18,
 2,
 5,
 62,
 386,
 12,
 8,
 316,
 8,
 106,
 5,
 4,
 2223,
 2,
 16,
 480,
 66,
 3785,
 33,
 4,
 130,
 12,
 16,
 38,
 619,
 5,
 25,
 124,
 51,
 36,
 135,
 48,
 25,
 1415,
 33,
 6,
 22,
 12,
 215,
 28,
 77,
 52,
 5,
 14,
 407,
 16,
 82,
 2,
 8,
 4,
 107,
 117,
 2,
 15,
 256,
 4,
 2,
 7,
 3766,
 5,
 723,
 36,
 71,
 43,
 530,
 476,
 26,
 400,
 317,
 46,
 7,
 4,
 2,
 1029,
 13,
 104,
 88,
 4,
 381,
 15,
 297,
 98,
 32,
 2071,
 56,
 26,
 141,
 6,
 194,
 2,
 18,
 4,
 226,
 22,
 21,
 134,
 476,
 26,
 480,
 5,
 144,
 30,
 2,
 18,
 51,
 36,
 

In [11]:
len(X_train[0])#first Row has 218 words

218

In [14]:
#len(X_train[1001])
#len(X_train[796])
#len(X_train[423])
#len(X_train[205])

181

### KP_Task: Create a function to see number of words in each Row/Sample?

In [3]:
top_words

5000

In [4]:
y_train

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

##### Performing padding to reduce length of each row to max 500 words(you can keep it any length)

In [15]:
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

### 4.Training using just LSTM

In [16]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))#Adding embedding layer
model.add(LSTM(100)) #added 100 neurons to RNN 
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
So Embedding layer solves 160000 weights, Embeding layer has 500 neurons
LSTM cell has 100 neurons, the number of PArameters it fixes is 53200

###### Training the LSTM model 

In [17]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

C:\Users\kishan\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 625s 25ms/step - loss: 0.4801 - accuracy: 0.7634 - val_loss: 0.3391 - val_accuracy: 0.8584
Epoch 2/3
25000/25000 [==============================] - 655s 26ms/step - loss: 0.3384 - accuracy: 0.8589 - val_loss: 0.3671 - val_accuracy: 0.8452
Epoch 3/3
25000/25000 [==============================] - 661s 26ms/step - loss: 0.2596 - accuracy: 0.8996 - val_loss: 0.3292 - val_accuracy: 0.8632


##### Thus after 1st epoch training accuracy: 0.76 and test accuracy is : 0.85# Not good
Thus after 2nd epoch training accuracy: 0.85 and test accuracy is : 0.84 # good
Thus after 1st epoch training accuracy: 0.89 and test accuracy is : 0.86 #good



In [18]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
#This is the best of all "validatoon accuracy"

Accuracy: 86.32%


###### This is the best of all "validation accuracy"

###  5. LSTM For Sequence Classification With Dropout

##### importing Libraries

In [19]:
import numpy
from keras.datasets import imdb #import imdb dataset
from keras.models import Sequential#cearate a sequential model
from keras.layers import Dense #ading a dense layer to RNN network
from keras.layers import LSTM #For LSTM 
from keras.layers import Dropout #To avoid overfitting, we useOverfitting
from keras.layers.embeddings import Embedding #To add embedding layer to RNN
from keras.preprocessing import sequence

In [20]:
'''model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))'''

##### Model structure/Schema

In [21]:
numpy.random.seed(1234)

# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))#drop out at embedded layer
model.add(LSTM(100))
model.add(Dropout(0.2))#drop out at LSTM layer
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\kishan\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
25000/25000 [==============================] - 512s 20ms/step - loss: 0.4762 - accuracy: 0.7541
Epoch 2/3
25000/25000 [==============================] - 533s 21ms/step - loss: 0.3117 - accuracy: 0.8742
Epoch 3/3
25000/25000 [==============================] - 536s 21ms/step - loss: 0.2923 - accuracy: 0.8813
Accuracy: 86.69%


##### The accuracy incrases from 86.32 % to 86.69% when Drop out introduced

#Keras provides this capability with parameters on the LSTM layer, the dropout for configuring the input dropout and recurrent_dropout for configuring the recurrent dropout.

###### Adding dropout also in the recurrent layer too
###### I is double drop out, Drop out in between latywers and drop out within LSTM layers

### 6. Double Dropout

In [22]:
# create the model with double drop out, drop out between layers and drop out within layers of LSTM
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


C:\Users\kishan\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
25000/25000 [==============================] - 582s 23ms/step - loss: 0.4956 - accuracy: 0.7590
Epoch 2/3
25000/25000 [==============================] - 609s 24ms/step - loss: 0.3852 - accuracy: 0.8369
Epoch 3/3
25000/25000 [==============================] - 584s 23ms/step - loss: 0.3589 - accuracy: 0.8499
Accuracy: 85.52%


### 7. LSTM and Convolutional Neural Network For Sequence Classification

In [23]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [24]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)
# truncate and pad input sequences
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)#to make each row of same length
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)#to make each row of same length

In [26]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 216,405
Trainable params: 216,405
Non-trainable params: 0
_________________________________________________________________
None


###### 7 a Training it 

In [27]:
model.fit(X_train, y_train, epochs=3, batch_size=64)

C:\Users\kishan\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
25000/25000 [==============================] - 258s 10ms/step - loss: 0.4167 - accuracy: 0.7966
Epoch 2/3
25000/25000 [==============================] - 266s 11ms/step - loss: 0.2433 - accuracy: 0.9060
Epoch 3/3
25000/25000 [==============================] - 269s 11ms/step - loss: 0.2003 - accuracy: 0.9248


In [28]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.08%


##### We can observe the accuracy increased as we added 1D CNN layer to our RNN 

### 8. RNN + CNN 1 D and Flattenlayer too

In [29]:
# introduce the concept of flatten and FCN - fully connected network to optimize the model further
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 500, 32)           3104      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 250, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 250)               2000250   
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 251       
Total params: 2,163,605
Trainable params: 2,163,605
Non-trainable params: 0
____________________________________________

In [30]:
model.fit(X_train, y_train, epochs=3, batch_size=128)# with modified Batch Size

C:\Users\kishan\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
25000/25000 [==============================] - 44s 2ms/step - loss: 0.4546 - accuracy: 0.7624
Epoch 2/3
25000/25000 [==============================] - 47s 2ms/step - loss: 0.2161 - accuracy: 0.9158
Epoch 3/3
25000/25000 [==============================] - 44s 2ms/step - loss: 0.1546 - accuracy: 0.9424


In [31]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 88.29%


#####  8 a)Changing the batch size and then training the model

In [32]:
model.fit(X_train, y_train, epochs=3, batch_size=64)

Epoch 1/3
25000/25000 [==============================] - 63s 3ms/step - loss: 0.1161 - accuracy: 0.9584
Epoch 2/3
25000/25000 [==============================] - 60s 2ms/step - loss: 0.0417 - accuracy: 0.9887
Epoch 3/3
25000/25000 [==============================] - 62s 2ms/step - loss: 0.0106 - accuracy: 0.9981


In [33]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.32%


In [34]:
model.predict_classes(X_test) # predicted sentiment for test set

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [35]:
y_test # actual sentiment for test set

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [ ]:
# 1 - positive
# 0 -  negative

### Conclusion:
    In this project i tried to perform Sentiment Analysis on a imdb movie dataset which consists of 25,000 Reviews for training and 25,000 reviews for testing.
    We started with simply 
        1)Loading the dataset
        2)Imorting required Libraries
        3)Preprocesing the Dataset to carry out further Analysis
        
    I Tried Padding to equalise each row to same length
    Used LSTM
    Also tried LSTM with dropout (to avoid Overfitting)
    Double Dropout (Droput in Layers and also in LSTM layer)
    LSTM wth CNN of 1 D
    LSTM wth CNN of 1 D and flatten layer
    Chaning Batch Size for understanding variation in accuracy
    
    Question:What else can be done?
    Answer: I can try to see the change in accuracy observed due to change in:
        1)Activation function
        2)Batch size during training
        3)Optimiser functions like :adam, sgd
        4)Adding or removing more hidden layers and Dense layers
        5)Adding or removing : Drop-Outs
        6)Adding Flatten layer
        7)changing Filter size in 1-D CNN
        8)Changing Embedded Vector Length
            